In [3]:
import numpy as np 
import glob
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import *
from keras.layers.merge import concatenate
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


# Data processing

In [5]:
# make config
out_rows = 256
out_cols = 256
train_path = "train"
label_path = "label"
img_type = "jpg"
test_path = "test"
npy_path = "npydata"

In [6]:
def create_data(data_path, npy_path):   
    imgs = glob.glob("data\\" + data_path + "\\*." + img_type)
    i = 0
    img_count = len(imgs)
    imgs_data = np.ndarray((img_count, out_rows, out_cols, 1), dtype=np.uint8)
    for img_full_name in imgs:
        img_name = img_full_name[img_full_name.rindex("\\") + 1:]
        # test
        img = load_img("data\\" + data_path + "\\" + img_name, grayscale = True)
        img = img_to_array(img)
        imgs_data[i] = img
        if i % 10 == 0:
            print('Done: {0}/{1} images'.format(i, img_count))
        i += 1
    print('Created.')
    np.save("data\\" + npy_path + '\\imgs_' + data_path + '.npy', imgs_data)
    print('Saving to .npy files done.')

In [7]:
def load_data(data_path, npy_path, process_colors = False):
    imgs = np.load("data\\" + npy_path + '\\imgs_' + data_path + '.npy')
    imgs = imgs.astype('float32')
    imgs /= 255
    if (process_colors):
        imgs[imgs > 0.5] = 1
        imgs[imgs <= 0.5] = 0
    return imgs

# U-net

In [ ]:
def get_model(rows, cols):

    inputs = Input((rows, cols, 1))
    merge_axis = -1

    conv1 = Conv2D(32, 3, padding='same', activation='relu')(inputs)
    conv1 = Conv2D(32, 3, padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, padding='same', activation='relu')(pool1)
    conv2 = Conv2D(64, 3, padding='same', activation='relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(64, 3, padding='same', activation='relu')(pool2)
    conv3 = Conv2D(64, 3, padding='same', activation='relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(128, 3, padding='same', activation='relu')(pool3)
    conv4 = Conv2D(128, 3, padding='same', activation='relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256, 3, padding='same', activation='relu')(pool4)

    up1 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Conv2D(256, 3, padding='same', activation='relu')(up1)
    conv6 = Conv2D(256, 3, padding='same', activation='relu')(conv6)
    merged1 = concatenate([conv4, conv6], axis=merge_axis)
    conv6 = Conv2D(256, 3, padding='same', activation='relu')(merged1)

    up2 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Conv2D(256, 3, padding='same', activation='relu')(up2)
    conv7 = Conv2D(256, 3, padding='same', activation='relu')(conv7)
    merged2 = concatenate([conv3, conv7], axis=merge_axis)
    conv7 = Conv2D(256, 3, padding='same', activation='relu')(merged2)

    up3 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Conv2D(128, 3, padding='same', activation='relu')(up3)
    conv8 = Conv2D(128, 3, padding='same', activation='relu')(conv8)
    merged3 = concatenate([conv2, conv8], axis=merge_axis)
    conv8 = Conv2D(128, 3, padding='same', activation='relu')(merged3)

    up4 = UpSampling2D(size=(2, 2))(conv8)
    conv9 = Conv2D(64, 3, padding='same', activation='relu')(up4)
    conv9 = Conv2D(64, 3, padding='same', activation='relu')(conv9)
    merged4 = concatenate([conv1, conv9], axis=merge_axis)
    conv9 = Conv2D(64, 3, padding='same', activation='relu')(merged4)

    conv10 = Conv2D(1, 3, padding='same', activation='sigmoid')(conv9)


    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

# Main

In [8]:
create_data(train_path, npy_path)
create_data(label_path, npy_path)
create_data(test_path, npy_path)

Done: 0/144 images
Done: 10/144 images
Done: 20/144 images
Done: 30/144 images
Done: 40/144 images
Done: 50/144 images
Done: 60/144 images
Done: 70/144 images
Done: 80/144 images
Done: 90/144 images
Done: 100/144 images
Done: 110/144 images
Done: 120/144 images
Done: 130/144 images
Done: 140/144 images
Created.
Saving to .npy files done.
Done: 0/144 images
Done: 10/144 images
Done: 20/144 images
Done: 30/144 images
Done: 40/144 images
Done: 50/144 images
Done: 60/144 images
Done: 70/144 images
Done: 80/144 images
Done: 90/144 images
Done: 100/144 images
Done: 110/144 images
Done: 120/144 images
Done: 130/144 images
Done: 140/144 images
Created.
Saving to .npy files done.
Done: 0/5 images
Created.
Saving to .npy files done.
